#### Import libraries and install BeautifulSoup

In [17]:
import pandas as pd
import numpy as np
import requests
import urllib

# install BeautifulSoup package
! easy_install beautifulsoup4
from bs4 import BeautifulSoup

Searching for beautifulsoup4
Best match: beautifulsoup4 4.6.0
Adding beautifulsoup4 4.6.0 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for beautifulsoup4
Finished processing dependencies for beautifulsoup4


#### Read the Wikipedia page and use BeautifulSoup to parse document

In [18]:
response = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html_doc = response.read()

soup = BeautifulSoup(html_doc, 'lxml')
t_head=soup.find_all('th',class_="")
print (t_head)
t_rows=soup.find_all('td')

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
</th>]


#### Obtain columns from table header and rows from table data

In [19]:
# obtain columns
cols=[]
for h in t_head: 
    cols.append(h.text.strip())

print(cols)
    
# obtain rows
rows=[]
subr=[]
i=1
for r in t_rows: 
    if i%3==0:
        subr.append(r.text.strip())
        rows.append(subr)
        subr=[]
    else:
        subr.append(r.text.strip())
    i = i + 1

print(rows[0:6])

['Postcode', 'Borough', 'Neighbourhood']
[['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park']]


#### Put data in a dataframe and clean data

In [39]:
# create dataframe
df = pd.DataFrame(rows, columns=cols)

# remove rows that are not part of the Postal Code table
df = df.drop(df.index[288:])

print('Initial shape: ' + str(df.shape))

# remove rows with a Borough that has a value of 'Not assigned'
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)

# if a Neighbourhood is Not assigned, then the Neighbourhood value will be the same as the Borough
df['Neighbourhood'] = df.apply(lambda row: row['Borough'] if (row['Neighbourhood']=='Not assigned') else row['Neighbourhood'], axis=1)

df.head(8)

Initial shape: (288, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue


#### Combine Neighborhoods by Borough to one row, with Neighborhoods separated with a comma.

In [40]:
# sort dataframe
df.sort_values('Postcode')

# combine Neighborhoods separated by comma
for i in range(0,len(df.index)-1): 
    if df.iloc[i][0] == df.iloc[i+1][0]:
        df.iloc[i+1][2] = df.iloc[i][2] + ', ' + df.iloc[i+1][2]

df.head(8)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,Lawrence Heights
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue


#### Group dataframe by Postcode and select unique rows

In [41]:
# select rows where the Neighborhood value length is the biggest
df = df.groupby(['Postcode','Borough'],as_index=False)['Neighbourhood'].max()

print('Final shape: ' + str(df.shape))

df

Final shape: (103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Get the latitude and the longitude coordinates of each neighborhood
#### (Tried to use GEOCODER and make it work, but I'm getting [REQUEST_DENIED])

In [23]:
# install and import geocoder
! pip install geocoder
import geocoder

# initialize your variable to None
lat_lng_coords = None
i=1

# loop until you get the coordinates
while(lat_lng_coords is None):
    #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    g = geocoder.google('Mountain View, CA')
    lat_lng_coords = g.latlng
    i=i+1
    if i==100:
        break

print(g)    
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: i

#### Will use the csv file with geographical coordinates instead

In [86]:
import io
import requests

url = "http://cocl.us/Geospatial_data"
s = requests.get(url).content
coord = pd.read_csv(io.StringIO(s.decode('utf-8')))
coord.set_index('Postal Code', inplace=True)
coord.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


#### Use the csv data to generate the  dataframe with coordinates

In [98]:
# create new columns
df_ll = df
df_ll['Latitude'] = ''
df_ll['Longitude'] = ''

print(df_ll.columns)

# lookup Postcode coordinates 
for i in range(0,len(df_ll)): 
    rowll= coord.loc[df['Postcode'][i]]
    df['Latitude'][i] = rowll[0]
    df['Longitude'][i] = rowll[1]
    
df_ll

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648
